# Hydraulic Stability Calculator for Breakwater Armor Units
### Concrete Cubes & Antifer Blocks

## 1. Abstract

This notebook implements computational tools designed for the hydraulic design of coastal breakwater armor layers. The software implements well-known empirical formulae to dimension artificial concrete units, specifically **Simple Cubes** (Van der Meer) and **Antifer Blocks** (Chegini & Aghtouman).

A distinguishing feature of this tool is its **Iso-Geometric Design Philosophy** for the breakwater head. Rather than increasing the nominal diameter ($D_n$) of armor units and/or softening the slope ($m:1$) at the roundhead—which necessitates different casting moulds, storage logistics, and crane requirements—this calculator solves for the required **increase in concrete density** ($\rho_c$).

This allows the Trunk and the Head to be constructed using geometrically identical units (same moulds) and identical slope angles, optimizing construction efficiency while meeting equal safety factors via material density adjustments.

## 2. Theoretical Framework & Methodology

The software calculates design parameters based on formulas that were calibrated as a result of hydraulic model test data and established coastal engineering standards.

### 2.1 Wave Mechanics & Geometric Parameters

Before determining armor stability, the software computes wave and structure properties:

**Deep Water Wavelength ($L_0$):**
Calculated based on the mean wave period ($T_m$) using linear wave theory approximation for deep water:
$$L_0 = \frac{g T_m^2}{2\pi}$$

**Wave Steepness ($s_{0m}$):**
A critical parameter influencing breaker type and stability:
$$s_{0m} = \frac{H_s}{L_0}$$

**Number of Waves ($N_z$):**
The total number of waves attacking the structure during the design storm duration ($t$ in hours):
$$N_z = \frac{t \times 3600}{T_m}$$

**Relative Buoyant Density ($\Delta$):**
The dimensionless density of the concrete relative to seawater:
$$\Delta = \frac{\rho_c}{\rho_w} - 1$$

### 2.2 Trunk Stability: Empirical Power Laws

The core of the calculator utilizes power-law formulas derived from extensive hydraulic model testing. The Stability Number ($N_s$) is the governing dimensionless parameter.

**General Stability Formula:**
$$N_s = \left( k_1 \frac{N_{od}^{k_2}}{N_z^{k_3}} + k_4 \right) s_{0m}^{-k_5}$$

*Where:*
* $N_{od}$ is the Damage Number (normalized damage level).
* $k_{1..5}$ are empirical coefficients specific to the block type and slope.

**Nominal Diameter ($D_n$):**
Once $N_s$ is determined, the required characteristic size of the block is:
$$D_n = \frac{H_s}{\Delta \cdot N_s}$$

**Armor Unit Weight ($W$):**
$$W = \rho_c \cdot D_n^3$$

#### Empirical Coefficients ($k$)

The software utilizes the following database of coefficients:

| Method | Block Type | Slope | $k_1$ | $k_2$ | $k_3$ | $k_4$ | $k_5$ |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| **Van der Meer (1988)** | Simple Cubes | 2.0:1 | 7.374 | 0.400 | 0.300 | 1.101 | 0.100 |
| **Van der Meer (1988)** | Simple Cubes | 1.5:1 | 6.700 | 0.400 | 0.300 | 1.000 | 0.100 |
| **Chegini (2006)** | Antifer | 2.0:1 | 6.138 | 0.443 | 0.276 | 1.164 | 0.070 |
| **Chegini (2006)** | Antifer | 1.5:1 | 6.951 | 0.443 | 0.291 | 1.082 | 0.082 |

### 2.3 The Head Design Strategy: Iso-Geometric Transfer

The breakwater head is subjected to 3D turbulence and wave breaking significantly higher than the trunk. Standard practice suggests increasing the block size and/or softening the slope. However, this tool uses a "Transfer Function" approach to maintain constant geometry ($D_n$ and Slope).

**Mathematical Derivation:**
1.  **Principle:** We equate the size ($D_n$) of the trunk and head, but solve for a higher density at the head to satisfy stability requirements.
2.  **Stability Ratio:** We enforce that the Head must be stable against higher forces, represented by a safety ratio of 1.5 in the stability coefficients.
    $$Ratio = \frac{K_{D,trunk}}{K_{D,head}} = 1.5$$
3.  **Transfer Function:** Since stability in the Hudson-like framework is proportional to $\Delta^3$, the required density increase is:
    $$\Delta_{head} = \Delta_{trunk} \cdot (1.5)^{1/3}$$

**Result:** The software outputs a required **High Density Concrete** ($\rho_{c,head}$) for the head units.

In [1]:
import math
import pandas as pd

# ==========================================
# PHYSICAL CONSTANTS
# ==========================================
G = 9.80665  # Acceleration due to gravity (m/s^2)
W_ROCK_SPEC = 26.5 # Specific weight for underlayer rock (kN/m3) (Basalt/Granite)

# Porosity (P)
P_CUBES = 0.40  # Armor layer
P_ROCK = 0.25   # Rock underlayer

# Safety Factor for Head Stability (Iso-Geometric Design)
KD_RATIO_FIXED = 1.5

## 3. Databases

### 3.1 Empirical Coefficients
Coefficients for Van der Meer (Cubes) and Chegini (Antifer) formulas.

In [2]:
FORMULAS = {
    1: {
        "name": "Van Der Meer (1988) - Cubes (Slope 2.0:1)",
        "type": "Cubes", "slope_ratio": 2.0, 
        "k1": 7.374, "k2": 0.4, "k3": 0.3, "k4": 1.101, "k5": 0.1
    },
    2: {
        "name": "Van Der Meer (1988) - Cubes (Slope 1.5:1)",
        "type": "Cubes", "slope_ratio": 1.5,
        "k1": 6.700, "k2": 0.4, "k3": 0.3, "k4": 1.000, "k5": 0.1
    },
    3: {
        "name": "Chegini-Aghtouman (2006) - Antifer (Slope 2:1)",
        "type": "Antifer", "slope_ratio": 2.0,
        "k1": 6.138, "k2": 0.443, "k3": 0.276, "k4": 1.164, "k5": 0.07
    },
    4: {
        "name": "Chegini-Aghtouman (2006) - Antifer (Slope 1.5:1)",
        "type": "Antifer", "slope_ratio": 1.5,
        "k1": 6.951, "k2": 0.443, "k3": 0.291, "k4": 1.082, "k5": 0.082
    }
}

STANDARD_GRADINGS = [
    # Coarse / Light Gradings
    {"name": "CP45/125",       "NLL_kg": 0.4,    "NUL_kg": 1.2},
    {"name": "CP63/180",       "NLL_kg": 1.2,    "NUL_kg": 3.8},
    {"name": "CP90/250",       "NLL_kg": 3.1,    "NUL_kg": 9.3},
    {"name": "CP45/180",       "NLL_kg": 0.4,    "NUL_kg": 1.2},
    {"name": "CP90/180",       "NLL_kg": 2.1,    "NUL_kg": 2.8},
    # Light Mass Armourstone (LMA)
    {"name": "LMA5-40",        "NLL_kg": 5,      "NUL_kg": 40},
    {"name": "LMA10-60",       "NLL_kg": 10,     "NUL_kg": 60},
    {"name": "LMA15-120",      "NLL_kg": 15,     "NUL_kg": 120},
    {"name": "LMA40-200",      "NLL_kg": 40,     "NUL_kg": 200},
    {"name": "LMA60-300",      "NLL_kg": 60,     "NUL_kg": 300},
    {"name": "LMA15-300",      "NLL_kg": 15,     "NUL_kg": 300},
    # Heavy Mass Armourstone (HMA)
    {"name": "HMA300-1000",    "NLL_kg": 300,    "NUL_kg": 1000},
    {"name": "HMA1000-3000",   "NLL_kg": 1000,   "NUL_kg": 3000},
    {"name": "HMA3000-6000",   "NLL_kg": 3000,   "NUL_kg": 6000},
    {"name": "HMA6000-10000",  "NLL_kg": 6000,   "NUL_kg": 10000},
    {"name": "HMA10000-15000", "NLL_kg": 10000,  "NUL_kg": 15000},
]

## 4. Calculation Engine

The class below encapsulates the logic for sizing the trunk and head armor units.

In [3]:
class BreakwaterCalculator:
    def __init__(self):
        # ----------------------------------------------------------------------
        # PHYSICAL CONSTANTS
        # ----------------------------------------------------------------------
        self.g = 9.80665  # Acceleration due to gravity (m/s^2)
        self.W_rock_spec = 26.5 # Standard Specific weight for underlayer rock (kN/m3) (Basalt/Granite)
        
        # ----------------------------------------------------------------------
        # LAYER CHARACTERISTICS
        # ----------------------------------------------------------------------
        # Porosity (P) is the percentage of void space in the armor layer.
        self.P_cubes = 0.40  # Porosity of armor layers (Standard for double-layer Cubes)
        self.P_rock = 0.25   # Porosity of rock underlayers (Standard approximation)
        
        # ----------------------------------------------------------------------
        # ISO-GEOMETRIC HEAD DESIGN FACTOR
        # ----------------------------------------------------------------------
        # The head of a breakwater experiences higher hydraulic loads (3D effects).
        # We define a fixed ratio between the stability coefficient (Kd) of the Trunk vs Head.
        # Ratio = Kd_trunk / Kd_head = 1.5
        # This implies the Head needs to be ~1.5x more stable than the Trunk.
        self.KD_RATIO_FIXED = 1.5

        # ----------------------------------------------------------------------
        # DEFAULT INPUTS
        # ----------------------------------------------------------------------
        self.defaults = {
            "Hs": 10.0,                # Significant Wave Height (m)
            "Tm": 13.0,                # Mean Wave Period (s)
            "Storm_Duration_hr": 12.0, # Duration of the design storm
            "Nod": 1.0,                # Damage Number (Nod=0 to 1 implies no damage/start of damage)
            "Wc": 24.0,                # Specific weight of concrete (kN/m3)
            "Ww": 10.05,               # Specific weight of seawater (kN/m3)
            "Formula_ID": 1            # Default is VdM 2.0 (ID 1)
        }
        # Link global databases to instance
        self.formulas = FORMULAS
        self.standard_gradings = STANDARD_GRADINGS

    def calculate_L0(self, Tm):
        """Calculates Deepwater Wavelength L0 = g * Tm^2 / (2 * pi)"""
        return (self.g * Tm**2) / (2 * math.pi)

    def calculate_underlayer_params(self, W_armor):
        """
        Helper to calculate underlayer parameters.
        Logic: Target rock is 1/10th the weight of armor. Finds the EN 13383 grading
        that strictly contains the target mass, breaking ties with the tightest range.
        """
        target_weight = W_armor / 10.0
        target_mass_kg = (target_weight * 1000.0) / self.g
        
        selected_grading = None
        min_range_width = float('inf')
        
        # --- CONTAINMENT & TIGHTEST RANGE LOGIC ---
        for grading in self.standard_gradings:
            # Check containment: Target must be strictly inside nominal limits
            if grading["NLL_kg"] < target_mass_kg < grading["NUL_kg"]:
                current_range = grading["NUL_kg"] - grading["NLL_kg"]
                
                # Update if this is the first match OR if this range is tighter (smaller)
                if current_range < min_range_width:
                    min_range_width = current_range
                    selected_grading = grading
                    
        # Fallback if no grading strictly contains the target mass
        if not selected_grading and self.standard_gradings:
            selected_grading = self.standard_gradings[0]

        final_NLL = selected_grading["NLL_kg"]
        final_NUL = selected_grading["NUL_kg"]
        final_M50 = 0.5 * (final_NLL + final_NUL)
        
        # --- EN 13383 LIMIT CALCULATIONS ---
        ELL = 0.7 * final_NLL
        EUL = 1.5 * final_NUL
        W_mean_kn = (final_M50 * self.g) / 1000.0
        
        Dn_rock = (W_mean_kn / self.W_rock_spec) ** (1/3.0)
        r2 = 2.0 * Dn_rock
        f2 = 100.0 * 2.0 * 1.0 * (1.0 - self.P_rock) / (Dn_rock**2)
        
        return {
            "grading_name": selected_grading["name"],
            "target_W": target_weight,
            "target_M50_kg": target_mass_kg,
            "M50_kg": final_M50,
            "NLL_kg": final_NLL,
            "NUL_kg": final_NUL,
            "ELL_kg": ELL,
            "EUL_kg": EUL,
            "W_mean_kn": W_mean_kn, 
            "Dn_rock": Dn_rock, 
            "r2": r2, 
            "f2": f2,
            "W_rock_spec": self.W_rock_spec
        }

    def solve(self, formula_id, user_inputs=None):
        # 1. Load Parameters (Merge defaults with any user overrides)
        params = self.defaults.copy()
        if user_inputs:
            params.update(user_inputs)
        
        # 2. Load Coefficients from the Formula Database
        if formula_id not in self.formulas:
            raise ValueError(f"Invalid Formula ID: {formula_id}")
        
        coeffs = self.formulas[formula_id]
        k1, k2, k3, k4, k5 = coeffs['k1'], coeffs['k2'], coeffs['k3'], coeffs['k4'], coeffs['k5']

        # 3. Preliminary Hydraulic Calculations
        # Calculate deep water wavelength and wave steepness (s0m)
        L0 = self.calculate_L0(params['Tm'])
        k0 = (2 * math.pi) / L0
        s0m = params['Hs'] / L0
        sm = s0m 

        # Calculate Number of Waves (Nz) during the storm
        Nz = (params['Storm_Duration_hr'] * 3600) / params['Tm']
        
        # Calculate Relative Density Delta = (Rho_concrete / Rho_water) - 1
        delta_trunk = (params['Wc'] / params['Ww']) - 1

        # 4. Algorithmic Core (Chegini-Aghtouman / Van der Meer) - TRUNK
        # This calculates the Stability Number (Ns)
        
        # Term 1: Damage / Waves relationship
        term_damage = params['Nod'] ** k2
        term_waves = Nz ** k3
        damage_wave_ratio = term_damage / term_waves
        
        # Combine terms based on power-law formula
        scaled_term = k1 * damage_wave_ratio
        inv_f = scaled_term + k4
        f_stab = 1.0 / inv_f 
        
        # Wave steepness influence
        steepness_factor = s0m ** (-k5)
        
        # Calculate Stability Number
        Ns_trunk = inv_f * steepness_factor

        # 5. Block Sizing (Armor) - TRUNK
        # Main Stability Equation: Dn = Hs / (Delta * Ns)
        Dn = params['Hs'] / (delta_trunk * Ns_trunk)
        
        # Weight = Specific_Weight * Volume (Dn^3)
        W_trunk = params['Wc'] * (Dn ** 3)
        
        # --- PACKING DENSITY CALCULATION (TRUNK) ---
        # Formula: 100 * Layers(2) * Coeff(1) * (1 - Porosity) / Dn^2
        packing_density_trunk = 100.0 * 2.0 * 1.1 * (1.0 - self.P_cubes) / (Dn**2)

        # 6. Hudson Comparative Calculation (Kd Trunk)
        # Calculates the equivalent Hudson Stability Coefficient (Kd) for reference.
        slope = coeffs['slope_ratio']
        kd_trunk_equiv = (params['Wc'] * (params['Hs']**3)) / (W_trunk * (delta_trunk**3) * slope)

        # 7. UNDERLAYER - TRUNK
        # Independently calculated based on Trunk Armor Weight
        ul_trunk = self.calculate_underlayer_params(W_trunk)

        # 8. HEAD DESIGN (Iso-Geometric Transfer Strategy)
        # -------------------------------------------------------------
        # STRATEGY: ISO-GEOMETRIC DESIGN PHILOSOPHY
        # Instead of increasing the block size (Dn) for the head, we keep
        # the size constant to match the Trunk cube size.
        # To achieve stability, we increase the density of the concrete.
        # -------------------------------------------------------------
        
        # Logic: Kd_trunk / Kd_head = 1.5
        kd_ratio = self.KD_RATIO_FIXED
        kd_head_derived = kd_trunk_equiv / kd_ratio
        
        # Calculate Delta Required for Head (to keep Dn same as trunk)
        # From Hudson: Kd ~ Delta^3. Therefore Delta_head = Delta_trunk * (Ratio)^(1/3)
        delta_head = delta_trunk * (kd_ratio**(1/3.0))
        
        # Convert required Delta back to Concrete Specific Weight (Wc)
        Wc_head = params['Ww'] * (delta_head + 1)
        
        # Head Weight: W_head = W_trunk * (Wc_head / Wc_trunk)
        # Note: W_head is usually > W_trunk due to higher density required
        W_head = W_trunk * (Wc_head / params['Wc'])

        # Calculate Stability Number for Head
        Ns_head = params['Hs'] / (delta_head * Dn)
        
        # --- PACKING DENSITY CALCULATION (HEAD) ---
        # Since Dn is maintained constant between Head and Trunk, 
        # the packing density (units/100m2) will actually be identical.
        # We perform the calculation again for verification.
        packing_density_head = 100.0 * 2.0 * 1.1 * (1.0 - self.P_cubes) / (Dn**2)

        # 9. UNDERLAYER - HEAD
        # Independently calculated based on Head Armor Weight (W_head)
        # Since W_head differs from W_trunk (heavier), this may result in a different grading.
        ul_head = self.calculate_underlayer_params(W_head)

        # 10. Armor Layer Details (Common)
        # r1 is the theoretical thickness of the double armor layer
        r1 = 2.0 * 1.1 * Dn

        # --- CALCULATE CUBE DIMENSIONS (H, A, B) ---
        # These shape factors are specific to the unit type (Cubes vs Antifer).
        # Note: 1.0247 is a shape factor approximation used here for volume.
        
        # Trunk Dimensions
        vol_trunk = W_trunk / params['Wc']
        h_trunk = (vol_trunk / 1.0247)**(1/3.0)
        a_trunk = 1.086 * h_trunk
        b_trunk = 1.005 * h_trunk

        # Head Dimensions
        vol_head = W_head / Wc_head
        h_head = (vol_head / 1.0247)**(1/3.0)
        a_head = 1.086 * h_head
        b_head = 1.005 * h_head

        # 11. Compile Results into a Dictionary
        results = {
            "inputs": params,
            "coefficients": coeffs,
            "intermediate": {
                "L0": L0, 
                "k0": k0,    # Added to results
                "s0m": s0m,  # Added to results
                "sm": sm, 
                "Nz": Nz, 
                "delta": delta_trunk, 
                "Ns_trunk": Ns_trunk 
            },
            "final_trunk": {
                "Dn": Dn,
                "W": W_trunk,
                "Mass_tonnes": W_trunk / self.g,
                "Kd_Equivalent": kd_trunk_equiv,
                "r1": r1,
                "packing_density": packing_density_trunk,
                "dims": {
                    "H": h_trunk,
                    "A": a_trunk,
                    "B": b_trunk
                }
            },
            "underlayer_trunk": ul_trunk,
            "final_head": {
                "Ns_head": Ns_head, 
                "Kd_Derived": kd_head_derived,
                "Kd_Ratio": kd_ratio,
                "Delta_Required": delta_head,
                "Wc_Required": Wc_head,
                "W": W_head,
                "Mass_tonnes": W_head / self.g,
                "packing_density": packing_density_head,
                "dims": {
                    "H": h_head,
                    "A": a_head,
                    "B": b_head
                }
            },
            "underlayer_head": ul_head,
            "constants": {
                "P_rock": self.P_rock,
                "P_cubes": self.P_cubes
            }
        }
        return results

    def print_report(self, results):
        # Unpack dictionary for easier string formatting
        p = results["inputs"]
        c = results["coefficients"]
        i = results["intermediate"]
        ft = results["final_trunk"]
        ut = results["underlayer_trunk"]
        fh = results["final_head"]
        uh = results["underlayer_head"]
        const = results["constants"]

        print("================================================================================")
        print("    TECHNICAL REPORT: BREAKWATER ARMOR & UNDERLAYER DESIGN                        ")
        print("================================================================================")
        print(f"Methodology: {c['name']}")
        print("-" * 80)
        
        # --- INPUT PARAMETERS ---
        print("1. INPUT PARAMETERS")
        print(f"   Hs (Sigificant Wave Height)         : {p['Hs']:.2f} m")
        print(f"   Tm (Mean Wave Period)               : {p['Tm']:.2f} s")
        print(f"   Storm Duration (h)                  : {p['Storm_Duration_hr']:.2f} h")
        print(f"   Nod (Damage)                        : {p['Nod']:.2f}")
        print(f"   Wc Trunk (Concrete Spec. Weight)    : {p['Wc']:.2f} kN/m3")
        print(f"   Ww (Water Specific Weight)          : {p['Ww']:.2f} kN/m3")
        print(f"   Relative Density D=(Wc/Ww)-1        : {i['delta']:.4f}")
        print(f"   Structure Slope (TRUNK & HEAD)      : {c['slope_ratio']}:1")
        print(f"   Porosity (Cubes)                    : {const['P_cubes']*100:.0f}%")
        print(f"   Porosity (Rock Layer)               : {const['P_rock']*100:.0f}%")
        print("-" * 80)
        
        # --- INTERMEDIATE PARAMETERS ---
        print("2. INTERMEDIATE PARAMETERS")
        print(f"   Wave Length (L0)                    : {i['L0']:.2f} m")
        print(f"   wave number (k0 = 2*pi/L0)          : {i['k0']:.4f}")
        print(f"   wave steepness (s0m = Hs/L0)        : {i['s0m']:.4f}")
        print(f"   Number of waves (Nz)                : {i['Nz']:.0f}")
        print(f"   Stability Number TRUNK (Ns)         : {i['Ns_trunk']:.4f}")
        print(f"   Stability Number HEAD (Ns)          : {fh['Ns_head']:.4f}")
        print("-" * 80)
        
        # --- TRUNK SECTION ---
        print("3. ARMOR LAYER RESULTS - TRUNK")
        print(f"   BLOCK WEIGHT (W)                    : {ft['W']:.2f} kN")
        print(f"   Mass (ton)                          : {ft['Mass_tonnes']:.2f} t")
        print(f"   Nominal Diameter (Dn)               : {ft['Dn']:.3f} m")
        print(f"   Cube Height (H)                     : {ft['dims']['H']:.3f} m")
        print(f"   Cube Top Width (B)                  : {ft['dims']['B']:.3f} m")
        print(f"   Cube Base Width (A)                 : {ft['dims']['A']:.3f} m")
        print(f"   KD_TRUNK (Equivalent)               : {ft['Kd_Equivalent']:.2f}")
        print(f"   Double Layer Thickness (r1)         : {ft['r1']:.2f} m")
        print(f"   Packing Density, d [units/100m2]    : {ft['packing_density']:.2f}")
        print("")
        
        # --- UNDERLAYER TRUNK ---
        print("4. UNDERLAYER RESULTS - TRUNK")
        print(f"   Theoretical Target (W/10)           : {ut['target_W']:.2f} kN ({ut['target_M50_kg']:.1f} kg)")
        print(f"   Adopted rock grading                : {ut['grading_name']}")
        print(f"   Representative M50                  : {ut['M50_kg']:.1f} kg")
        print(f"   Nominal lower limit (NLL)           : {ut['NLL_kg']:.1f} kg")
        print(f"   Nominal upper limit (NUL)           : {ut['NUL_kg']:.1f} kg")
        print(f"   Extreme lower limit (ELL)           : {ut['ELL_kg']:.1f} kg")
        print(f"   Extreme upper limit (EUL)           : {ut['EUL_kg']:.1f} kg")
        print(f"   Nominal Diameter (Dn_rock)          : {ut['Dn_rock']:.3f} m")
        print(f"   Double Layer Thickness (r2)         : {ut['r2']:.2f} m")
        print(f"   Packing Density, f2 [rocks/100m2]   : {ut['f2']:.2f}")
        print("-" * 80)

        # --- HEAD SECTION ---
        print("5. ARMOR LAYER RESULTS - HEAD (Iso-Geometric / High Density)")
        print("   *Maintains same Dn and Slope as Trunk*")
        print(f"   Stability Ratio (Kd_T/Kd_H)         : {fh['Kd_Ratio']:.2f}")
        print(f"   Nominal Diameter (Dn)               : {ft['Dn']:.3f} m")
        print(f"   Cube Height (H)                     : {fh['dims']['H']:.3f} m")
        print(f"   Cube Top width (B)                  : {fh['dims']['B']:.3f} m")
        print(f"   Cube Base Width (A)                 : {fh['dims']['A']:.3f} m")
        print(f"   KD_HEAD (Equivalent)                : {fh['Kd_Derived']:.2f}")
        print(f"   Required Concrete Density (Wc)      : {fh['Wc_Required']:.2f} kN/m3")
        print(f"   BLOCK WEIGHT (W)                    : {fh['W']:.2f} kN")
        print(f"   Mass (ton)                          : {fh['Mass_tonnes']:.2f} t")
        print(f"   Packing Density, d [units/100m2]    : {fh['packing_density']:.2f}")
        print("")
        
        # --- UNDERLAYER HEAD ---
        print("6. UNDERLAYER RESULTS - HEAD")
        print(f"   Theoretical Target (W/10)           : {uh['target_W']:.2f} kN ({uh['target_M50_kg']:.1f} kg)")
        print(f"   Adopted rock grading                : {uh['grading_name']}")
        print(f"   Representative M50                  : {uh['M50_kg']:.1f} kg")
        print(f"   Nominal lower limit (NLL)           : {uh['NLL_kg']:.1f} kg")
        print(f"   Nominal upper limit (NUL)           : {uh['NUL_kg']:.1f} kg")
        print(f"   Extreme lower limit (ELL)           : {uh['ELL_kg']:.1f} kg")
        print(f"   Extreme upper limit (EUL)           : {uh['EUL_kg']:.1f} kg")
        print(f"   Nominal Diameter (Dn_rock)          : {uh['Dn_rock']:.3f} m")
        print(f"   Double Layer Thickness (r2)         : {uh['r2']:.2f} m")
        print(f"   Packing Density, f2 [rocks/100m2]   : {uh['f2']:.2f}")
        print("================================================================================", end="")

## 3. Input Parameters

| Parameter | Symbol | Unit | Description |
| :--- | :---: | :---: | :--- |
| **Sig. Wave Height** | $H_s$ | meters | The average height of the highest 1/3 of waves at the toe of the structure. |
| **Mean Period** | $T_m$ | seconds | The average wave period. |
| **Damage Number** | $N_{od}$ | - | Allowable damage level. <br> $N_{od}=0$: No damage <br> $N_{od}=0.5$: Start of damage <br> $N_{od}=2.0$: Severe damage |
| **Storm Duration** | $t$ | hours | Duration of the peak design storm event (determines $N_z$). |
| **Concrete Density** | $\rho_c$ | kN/m³ | Specific weight of the concrete used in the Trunk. |
| **Formula ID** | - | - | Selects between Van der Meer (Cubes) or Chegini (Antifer) and slope variations. |

In [4]:
# ==============================================================================
# INPUT PARAMETERS
# ==============================================================================
# Modify these values to change the design conditions.

inputs = {
    "Hs": 10.0,                # Significant Wave Height (m)
    "Tm": 13.0,                # Mean Wave Period (s)
    "Storm_Duration_hr": 12.0, # Duration of the design storm
    "Nod": 1.0,                # Damage Number (0=No damage, 2=Severe)
    "Wc": 24.0,                # Specific weight of concrete (kN/m3)
    "Ww": 10.05,               # Specific weight of seawater (kN/m3)
}

# Formula Options:
# 1. Van Der Meer (1988) - Cubes (Slope 2.0:1)
# 2. Van Der Meer (1988) - Cubes (Slope 1.5:1)
# 3. Chegini-Aghtouman (2006) - Antifer (Slope 2.0:1)
# 4. Chegini-Aghtouman (2006) - Antifer (Slope 1.5:1)
formula_id = 1

# ==============================================================================
# EXECUTION
# ==============================================================================

# Initialize Calculator
calc = BreakwaterCalculator()

# Run Calculation
try:
    results = calc.solve(formula_id, inputs)
    calc.print_report(results)
except Exception as e:
    print(f"Calculation Error: {e}")

    TECHNICAL REPORT: BREAKWATER ARMOR & UNDERLAYER DESIGN                        
Methodology: Van Der Meer (1988) - Cubes (Slope 2.0:1)
--------------------------------------------------------------------------------
1. INPUT PARAMETERS
   Hs (Sigificant Wave Height)         : 10.00 m
   Tm (Mean Wave Period)               : 13.00 s
   Storm Duration (h)                  : 12.00 h
   Nod (Damage)                        : 1.00
   Wc Trunk (Concrete Spec. Weight)    : 24.00 kN/m3
   Ww (Water Specific Weight)          : 10.05 kN/m3
   Relative Density D=(Wc/Ww)-1        : 1.3881
   Structure Slope (TRUNK & HEAD)      : 2.0:1
   Porosity (Cubes)                    : 40%
   Porosity (Rock Layer)               : 25%
--------------------------------------------------------------------------------
2. INTERMEDIATE PARAMETERS
   Wave Length (L0)                    : 263.77 m
   wave number (k0 = 2*pi/L0)          : 0.0238
   wave steepness (s0m = Hs/L0)        : 0.0379
   Number of waves (Nz)

## 6. Bibliographic References

1.  **Van der Meer, J.W. (1988).** *Rock Slopes and Gravel Beaches Under Wave Attack.* Doctoral Thesis, Delft University of Technology.
2.  **Van der Meer, J.W. (1988).** "Stability of Cubes, Tetrapods and Accropode." *Proceedings of the Conference Breakwaters '88*, Eastbourne, Thomas Telford.
3.  **Chegini, V., & Aghtouman, P. (2006).** "An Investigation on the Stability of Rubble Mound Breakwaters with Armour Layers of Antifer Cubes." *Journal of Marine Engineering*.
4.  **USACE (2006).** *Coastal Engineering Manual (CEM)*, Chapter VI-5.
5.  **CEN (2002).** *EN 13383-1: Armourstone - Part 1: Specification*.